In [1]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, gwPll):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --gatewayReceptionPaths="+f"{gwPll}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os

def run_scenario(traffic_type, realistic_model, device_class, scenario_name, gwPll):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, gwPll)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_gw_parallel_paths'
    test_sub_folder = str(gwPll)

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []


In [7]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 1)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 2)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 3)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 4)))

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 8)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 16)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 32)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", 64)))


threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 1)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 2)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 3)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 4)))

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 8)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 16)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 32)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", 64)))

threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 1)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 2)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 3)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 4)))


threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 8)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 16)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 32)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", 64)))

In [8]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 1)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 2)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 3)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 4)))

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 1)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 2)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 3)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 4)))

threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 1)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 2)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 3)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 4)))



threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 8)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 16)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 32)))
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", 64)))

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 8)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 16)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 32)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", 64)))

threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 8)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 16)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 32)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", 64)))

In [9]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status GW_Parallel_Paths', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=1 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=2 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_B, Simualtion with device count: 2
ULPDR_UNCONFIRMED_B, Simualtion with 

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=3 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"ULPDR_UNCONFIRMED_C, Simualtion with device count: 2
ULPDR_UNCONFIRMED_B, Simualtion with device count: 3

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=4 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=4 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=1 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --rea

+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=8 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=1 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=3 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=4 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 2
ULPDR_CONFIRMED_B, Simualtion with device count: 3
ULPDR_CONFIRMED_B, Simualtion with device count: 4
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayReceptionPaths=16 --endDeviceType=B --appPeriod=600 --simulationTime

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
0.9899497487437185 L: 0.24875
1.0 L: 0.30375
0.9935483870967742 L: 0.3875
1.0 L: 0.45375
0.9866666666666667 L: 0.5625
0.9716814159292035 L: 0.70625
0.9658119658119658 L: 0.8775
0.9484173505275498 L: 1.06625
0.9135338345864662 L: 1.33
0.8633257403189066 L: 1.64625
0.8051001821493625 L: 2.05875
0.7232186136694134 L: 2.57875
0.6429693637077769 L: 3.1825
0.5582860743541272 L: 3.9675
0.45424420086668366 L: 4.90375
0.36082901554404145 L: 6.03125
0.2835970024979184 L: 7.50625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
0.9899497487437185 L: 0.24875
1.0 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9844444444444445 L: 0.5625
0.9787610619469026 L: 0.70625
0.9857549857549858 L: 0.8775
0.9742086752637749 L: 1.06625
0.9755639097744361 L: 1.33
0.9582384206529992 L: 1.64625
0.9216757741347905 L: 2.05875
0.900630150266602 L: 2.57875
0.8546739984289081 L: 3.1825
0.8037177063642093 L: 3.9675
0.7392301809839409 L: 4.90375
0.6726745390511705 L: 6.03375
0.5960033305578685 L: 7.50625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
0.9895833333333334 L: 0.12
1.0 L: 0.15125
0.9735099337748344 L: 0.18875
0.9698492462311558 L: 0.24875
0.9629629629629629 L: 0.30375
0.9548387096774194 L: 0.3875
0.928374655647383 L: 0.45375
0.92 L: 0.5625
0.8601769911504424 L: 0.70625
0.8105413105413105 L: 0.8775
0.7479484173505275 L: 1.06625
0.6917293233082706 L: 1.33
0.6006074411541382 L: 1.64625
0.5185185185185185 L: 2.05875
0.43092583616093066 L: 2.57875
0.35899450117831894 L: 3.1825
0.2892249527410208 L: 3.9675
0.22941626306398163 L: 4.90375
0.1669774186865548 L: 6.03375
0.12223147377185678 L: 7.50625
3136.161818742752
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
